In [ ]:
import math
import json
import os
import random
from IPython import display
import sympy as sp
import numpy as np
import pandas as pd
import sklearn

pd.set_option('display.max_rows', None)


%run config.py

Utility functions

In [ ]:
# Colors for terminal output
GREEN = "\033[92m"
RED = "\033[91m"
YELLOW = "\033[93m"
BLUE = "\033[94m"
CYAN = "\033[96m"
MAGENTA = "\033[95m"
BOLD = "\033[1m"
UNDERLINE = "\033[4m"
STANDARD = "\033[0m"

In [ ]:
# Sequencer and Desequencer
def csv2sequence(csvFile,seqFile,pref):
	with open(csvFile, 'r') as f:
		lines = f.readlines()
		with open(seqFile, 'w') as g:
			for line in lines:
				for token in line.split(','):
					if token.startswith(pref):
						token = token[len(pref):]
					token = token.strip()
					if token!="" and token!='\n':
						g.write(token+'\n')

def sequence2csv(seqFile,csvFile,dataWidth,pref):
	with open(seqFile, 'r') as f:
		lines = f.readlines()
		dataW = dataWidth
		with open(csvFile, 'w') as g:
			for line in lines:
				g.write(pref+line.strip())
				dataW -= 1
				if dataW==0:
					g.write('\n')
					dataW = dataWidth
				else:
					g.write(',')

In [ ]:
# Function to show the generated function
def showFunction():
	exec(open("generated.py").read())
	import IPython.display as disp
	disp.display(spExpr)
# showFunction()

In [ ]:
# Function to show the generated BM
def showBM():
	display.Image("bondmachine.png")

In [ ]:
def highlightDone(val):
    color = 'green' if val > 0.0 else 'red'
    return f'background-color: {color}'

environment creation functions

In [ ]:
# Function create an environment with a random mathematical expression
def generateRandom():
	! echo "SOURCE_FLEXPY=generated.py" > source.mk
	! make clean > /dev/null
	! echo "SOURCE_FLEXPY=generated.py" > source.mk
	! flexpytester --generate -e symbols -s generated.py -i inputs.csv -o outputs.csv --prefix --config decayFactor=3 > /dev/null
	! bmhelper apply > /dev/null 
	! make show > /dev/null
	csv2sequence('inputs.csv','inputs.seq',"0f")
	csv2sequence('outputs.csv','outputs.seq',"0f")
# generateRandom()

In [ ]:
# Function to generate the test cases of an existing mathematical expression
def generateExistent():
	! echo "SOURCE_FLEXPY=generated.py" > source.mk
	! make clean > /dev/null
	! echo "SOURCE_FLEXPY=generated.py" > source.mk
	! flexpytester --compute -e generated.py -i inputs.csv -o outputs.csv --prefix > /dev/null
	! bmhelper apply > /dev/null 
	! make show > /dev/null
	csv2sequence('inputs.csv','inputs.seq',"0f")
	csv2sequence('outputs.csv','outputs.seq',"0f")
# generateExistent()

In [ ]:
# Function to generate the test cases of an existing single neuron within the library
def generateNeuron(library, neuron, seq):
	! echo "SOURCE_BASM=out.basm" > source.mk
	! make clean > /dev/null
	! echo "SOURCE_BASM=out.basm" > source.mk
	! rm -f notok
	! fragtester -neuron-lib-path {library} -fragment-file {neuron}.basm -save-expression generated.py -seq {seq} -create-bmapi bmapi.json -build-app -app-flavor cpynqapi -app-file expression.c || touch notok > /dev/null
	if os.path.exists("notok"):
		return False
	! flexpytester --compute -e generated.py -i inputs.csv -o outputs.csv --prefix > /dev/null
	! bmhelper apply > /dev/null
	! make show > /dev/null
	csv2sequence('inputs.csv','inputs.seq',"0f")
	csv2sequence('outputs.csv','outputs.seq',"0f")
	return True
# generateNeuron("library", "cosargreal", "0")

In [ ]:
# Function to generate the test cases of an existing single neuron within the library
def getNeuronSeq(library, neuron):
	! rm -f notok
	! fragtester -neuron-lib-path {library} -fragment-file {neuron}.basm -save-expression generated.py  || touch notok > /dev/null
	if os.path.exists("notok"):
		return "0"
	! fragtester -neuron-lib-path {library} -fragment-file {neuron}.basm -save-expression generated.py -describe | grep "Sequences" | cut -d' ' -f2 > seqs.txt
	seq=1
	with open("seqs.txt", "r") as f:
		lines = f.readlines()
		for line in lines:
			seq=line.strip()
			return seq

In [ ]:
# Function to generate the test cases of an existing single neuron within the library
def placeHolderNeuron(library, neuron):
	! echo "{" > statistics.json
	! echo -n "\"" >> statistics.json
	! echo -n {neuron} >> statistics.json
	! echo "\": 0" >> statistics.json
	! echo "}" >> statistics.json

 Simulation beckends

In [ ]:
# BondMachine internal simulator
def runSimbatch():
	! make simbatch > /dev/null
# runSimbatch()

In [ ]:
# BondMachine hardware run
def runBMsim():
	! make deploycollect > /dev/null
	! mv working_dir/bmsim_outputs.seq bmsim_outputs.seq
	! bondmachine -bondmachine-file working_dir/bondmachine.json -list-outputs | wc -l > num_outputs.txt
	# Read the number of outputs
	with open('num_outputs.txt','r') as f:
		num_outputs = int(f.read())
	sequence2csv('bmsim_outputs.seq','bmsim_outputs.csv',num_outputs,"")
# runBMsim()

In [ ]:
# HLS simulator
def runHLSsim():
	print ("hlsim")
# runHLSsim()

Analysis

In [ ]:
def analyzeSimbatch():
	# Load the target outputs from the output.csv file
	targetData = np.loadtxt('outputs.csv', delimiter=',')
	simbatchData = np.loadtxt('simbatch_outputs.csv', delimiter=',')
	simbatchMSE=sklearn.metrics.mean_squared_error(targetData, simbatchData)

	data = {
		"Dataset": ["Sympy target", "Simbatch"],
		"MSE" : [0.0, simbatchMSE]
	}
	df = pd.DataFrame(data)
	return df.style.hide(axis="index"), simbatchMSE

	

In [ ]:
def analyzeBMsim():
	# Load the target outputs from the output.csv file
	targetData = np.loadtxt('outputs.csv', delimiter=',')
	bmsimData = np.loadtxt('bmsim_outputs.csv', delimiter=',')
	bmsimMSE=sklearn.metrics.mean_squared_error(targetData, bmsimData)

	data = {
		"Dataset": ["Sympy target", "BMsim"],
		"MSE" : [0.0, bmsimMSE]
	}
	df = pd.DataFrame(data)
	return df.style.hide(axis="index"), bmsimMSE

Statistics management functions

In [ ]:
def plotHistogram(df):
	# Plot the histogram
	df['bsimoccurrences'].plot(kind='bar', figsize=(10, 6), color='skyblue')


In [ ]:
def loadBsimRun(mse):
	with open('statistics.json', 'r') as f:
		data = json.load(f)
	
	df = pd.DataFrame.from_dict(data, orient='index', columns=['bsimoccurrences'])
	totOccurrences = df['bsimoccurrences'].sum()
	newCol=mse
	if totOccurrences > 0:
		newCol = newCol/totOccurrences
		df['bsimerror'] = newCol
	else:
		df['bsimerror'] = 0.0
	return df

In [ ]:
def loadBMsimRun(mse):
	with open('statistics.json', 'r') as f:
		data = json.load(f)
	
	df = pd.DataFrame.from_dict(data, orient='index', columns=['bmoccurrences'])
	totOccurrences = df['bmoccurrences'].sum()
	newCol=mse
	if totOccurrences > 0:
		newCol = newCol/totOccurrences
		df['bmerror'] = newCol
	else:
		df['bmerror'] = 0.0
	return df

In [ ]:
def PatchBsimGlobalStats(global_stats, df):
	# If the global statistics DataFrame is empty, initialize it with the same rows as the current DataFrame
	for index, row in df.iterrows():
		if index not in global_stats.index:
			global_stats.loc[index] = [0.0, 0.0, 0.0, 0.0]

	# Sum the occurrences to the global statistics
	for index, row in df.iterrows():
		global_stats.at[index, 'bsimoccurrences'] += row['bsimoccurrences']
		global_stats.at[index, 'bsimerror'] += row['bsimerror']
	# Save the updated global statistics
	global_stats.to_csv('global_statistics')
	return global_stats


In [ ]:
def PatchBMsimGlobalStats(global_stats, df):
	# If the global statistics DataFrame is empty, initialize it with the same rows as the current DataFrame
	for index, row in df.iterrows():
		if index not in global_stats.index:
			global_stats.loc[index] = [0.0, 0.0, 0.0, 0.0]

	# Sum the occurrences to the global statistics
	for index, row in df.iterrows():
		global_stats.at[index, 'bmoccurrences'] += row['bmoccurrences']
		global_stats.at[index, 'bmerror'] += row['bmerror']
	# Save the updated global statistics
	global_stats.to_csv('global_statistics')
	return global_stats


In [ ]:
def loadGlobalStats():
	# Load the global statistics from the CSV file if it exists
	if os.path.exists('global_statistics'):
		global_stats = pd.read_csv('global_statistics', index_col=0)
		global_stats.sort_index(inplace=True)
	else:
		# Create an empty DataFrame with the same columns as the global statistics
		global_stats = pd.DataFrame(columns=['bsimoccurrences', 'bsimerror', 'bmoccurrences', 'bmerror'])
	return global_stats

Interactive sessions

In [ ]:
# From here, it is possible to override the config.py settings and run specific parts

# singleRun=True
# singleRunPatch=True
fullRun=True
runSimbatchTests=True
# runBmsimTests=True

neuronUnderTest = "multargimagnumreal"
neuronSeq = "0"

In [ ]:
# Single run test, it executes the entire process and return a DataFrame (df) with the errors
if singleRun:	
	# generateRandom()
	# generateExistent()
	generateNeuron("library", neuronUnderTest, neuronSeq)

	if runSimbatchTests:
		runSimbatch()
		errors,mse=analyzeSimbatch()
		dfbsim=loadBsimRun(mse)

	if runBmsimTests:
		runBMsim()
		errors,mse=analyzeBMsim()
		dfbm=loadBMsimRun(mse)
	
	# runHLSsim()

In [ ]:
dfbsim if (singleRun and runSimbatchTests) else None

In [ ]:
dfbm if (singleRun and runBmsimTests) else None

In [ ]:
display.Image("bondmachine.png") if singleRun else None

In [ ]:
# Load the historical global statistics and patch it with the new data (df) with the simbatch results
if singleRunPatch and runBatchsimTests:
	gs=loadGlobalStats()
	gs=PatchBsimGlobalStats(gs, dfbsim)
	gs

In [ ]:
# Load the historical global statistics and patch it with the new data (df) with the bmsim results
if singleRunPatch and runBmsimTests:
	gs=loadGlobalStats()
	gs=PatchBMsimGlobalStats(gs, dfbm)
	gs

In [ ]:
gs if singleRunPatch else None

In [ ]:
if fullRun:
	# Iterate al the neurons in the library and test them singly, patching the global statistics
	gs=loadGlobalStats()

	for neuronFile in os.listdir("library"):
		if not neuronFile.endswith(".basm"):
			continue
		neuron = neuronFile.split(".")[0]
		# generateRandom()
		# generateExistent()

		# Simbatch tests
		if runSimbatchTests and not(any(gs.index.str.startswith(neuron)) and gs.loc[gs.index.str.startswith(neuron), 'bsimoccurrences'].max() > 0):
			seq=getNeuronSeq("library", neuron)
			if seq!="0":
				for i in range(0, int(seq)):
					if generateNeuron("library", neuron, str(i)):
						runSimbatch()
						errors,mse=analyzeSimbatch()
						df=loadBsimRun(mse)
						gs=PatchBsimGlobalStats(gs, df)
					else:
						print(f"{RED}Error generating neuron {neuron} with sequence {seq}{STANDARD}")
			else:
				placeHolderNeuron("library", neuron)

				df=loadBsimRun(0.0)
				gs=PatchBsimGlobalStats(gs, df)

		# BMsim tests
		if runBmsimTests and not(any(gs.index.str.startswith(neuron)) and gs.loc[gs.index.str.startswith(neuron), 'bmoccurrences'].max() > 0):
			seq=getNeuronSeq("library", neuron)
			if seq!="0":
				for i in range(0, int(seq)):
					if generateNeuron("library", neuron, str(i)):
						runBMsim()
						errors,mse=analyzeBMsim()
						df=loadBMsimRun(mse)
						gs=PatchBMsimGlobalStats(gs, df)
					else:
						print(f"{RED}Error generating neuron {neuron} with sequence {seq}{STANDARD}")
			else:
				placeHolderNeuron("library", neuron)

				df=loadBMsimRun(0.0)
				gs=PatchBMsimGlobalStats(gs, df)

	gs.style.format('{:.10f}', subset=['bsimerror'])
	gs.style.format('{:.10f}', subset=['bmerror'])


In [ ]:
gs.style.apply(lambda x: x.map(highlightDone), axis=None, subset=['bsimoccurrences', 'bmoccurrences']) if fullRun else None